# Maximal extension of the extremal Kerr black hole

This Jupyter/SageMath notebook is relative to the lectures
[Geometry and physics of black holes](https://luth.obspm.fr/~luthier/gourgoulhon/bh16/).

The computations make use of tools developed through the [SageManifolds project](https://sagemanifolds.obspm.fr).

version()

First we set up the notebook to display mathematical objects using LaTeX rendering:

In [1]:
%display latex

To speed up computations, we ask for running them in parallel on 8 threads:

In [2]:
Parallelism().set(nproc=8)

## Spacetime manifold

We declare the Kerr spacetime as a 4-dimensional Lorentzian manifold $M$:

In [3]:
M = Manifold(4, 'M', structure='Lorentzian')
print(M)

4-dimensional Lorentzian manifold M


We then introduce (3+1 version of) the **Kerr coordinates** $(\tilde{t},r,\theta,\tilde{\varphi})$ as a chart `KC` on $M$, via the method `chart()`. The argument of the latter is a string (delimited by `r"..."` because of the backslash symbols) expressing the coordinates names, their ranges (the default is $(-\infty,+\infty)$) and their LaTeX symbols:

In [4]:
KC.<tt,r,th,tph> = M.chart(r"tt:\tilde{t} r th:(0,pi):\theta tph:(0,2*pi):periodic:\tilde{\varphi}") 
print(KC); KC

Chart (M, (tt, r, th, tph))


Chart (M, (tt, r, th, tph))

In [5]:
KC.coord_range()

tt: (-oo, +oo); r: (-oo, +oo); th: (0, pi); tph: [0, 2*pi] (periodic)

## Metric tensor 

The mass parameter $m$ of the extremal Kerr spacetime is declared as a symbolic variable:

In [6]:
m = var('m', domain='real')
assume(m>0)

We get the (yet undefined) spacetime metric:

In [7]:
g = M.metric()

and initialize it by providing its components in the coordinate frame associated with the Kerr coordinates, which is the current manifold's default frame:

In [8]:
rho2 = r^2 + (m*cos(th))^2
g[0,0] = - (1 - 2*m*r/rho2)
g[0,1] = 2*m*r/rho2
g[0,3] = -2*m^2*r*sin(th)^2/rho2
g[1,1] = 1 + 2*m*r/rho2
g[1,3] = -m*(1 + 2*m*r/rho2)*sin(th)^2
g[2,2] = rho2
g[3,3] = (r^2 + m^2 + 2*m^3*r*sin(th)^2/rho2)*sin(th)^2
g.display()

g = (2*m*r/(m^2*cos(th)^2 + r^2) - 1) dtt*dtt + 2*m*r/(m^2*cos(th)^2 + r^2) dtt*dr - 2*m^2*r*sin(th)^2/(m^2*cos(th)^2 + r^2) dtt*dtph + 2*m*r/(m^2*cos(th)^2 + r^2) dr*dtt + (2*m*r/(m^2*cos(th)^2 + r^2) + 1) dr*dr - m*(2*m*r/(m^2*cos(th)^2 + r^2) + 1)*sin(th)^2 dr*dtph + (m^2*cos(th)^2 + r^2) dth*dth - 2*m^2*r*sin(th)^2/(m^2*cos(th)^2 + r^2) dtph*dtt - m*(2*m*r/(m^2*cos(th)^2 + r^2) + 1)*sin(th)^2 dtph*dr + (2*m^3*r*sin(th)^2/(m^2*cos(th)^2 + r^2) + m^2 + r^2)*sin(th)^2 dtph*dtph

A matrix view of the components with respect to the manifold's default vector frame:

In [9]:
g[:]

[                                2*m*r/(m^2*cos(th)^2 + r^2) - 1                                     2*m*r/(m^2*cos(th)^2 + r^2)                                                               0                        -2*m^2*r*sin(th)^2/(m^2*cos(th)^2 + r^2)]
[                                    2*m*r/(m^2*cos(th)^2 + r^2)                                 2*m*r/(m^2*cos(th)^2 + r^2) + 1                                                               0                  -m*(2*m*r/(m^2*cos(th)^2 + r^2) + 1)*sin(th)^2]
[                                                              0                                                               0                                             m^2*cos(th)^2 + r^2                                                               0]
[                       -2*m^2*r*sin(th)^2/(m^2*cos(th)^2 + r^2)                  -m*(2*m*r/(m^2*cos(th)^2 + r^2) + 1)*sin(th)^2                                                               0 (2*m^3*r*sin(th)^2/(m^2*cos(th)^2 + r^2) + m^2 + r^2)*sin(th)^2]

The list of the non-vanishing components:

In [10]:
g.display_comp()

g_tt,tt = 2*m*r/(m^2*cos(th)^2 + r^2) - 1 
g_tt,r = 2*m*r/(m^2*cos(th)^2 + r^2) 
g_tt,tph = -2*m^2*r*sin(th)^2/(m^2*cos(th)^2 + r^2) 
g_r,tt = 2*m*r/(m^2*cos(th)^2 + r^2) 
g_r,r = 2*m*r/(m^2*cos(th)^2 + r^2) + 1 
g_r,tph = -m*(2*m*r/(m^2*cos(th)^2 + r^2) + 1)*sin(th)^2 
g_th,th = m^2*cos(th)^2 + r^2 
g_tph,tt = -2*m^2*r*sin(th)^2/(m^2*cos(th)^2 + r^2) 
g_tph,r = -m*(2*m*r/(m^2*cos(th)^2 + r^2) + 1)*sin(th)^2 
g_tph,tph = (2*m^3*r*sin(th)^2/(m^2*cos(th)^2 + r^2) + m^2 + r^2)*sin(th)^2

Let us check that we are dealing with a solution of the **vacuum Einstein equation**:

In [11]:
#g.ricci().display()

## Regions $M_{\rm I}$ and $M_{\rm III}$

In [12]:
M_I = M.open_subset('M_I', latex_name=r'M_{\rm I}', coord_def={KC: r>m})
KC.restrict(M_I).coord_range()

tt: (-oo, +oo); r: (m, +oo); th: (0, pi); tph: [0, 2*pi] (periodic)

In [13]:
M_III = M.open_subset('M_III', latex_name=r'M_{\rm III}', coord_def={KC: r<m})
KC.restrict(M_III).coord_range()

tt: (-oo, +oo); r: (-oo, m); th: (0, pi); tph: [0, 2*pi] (periodic)

## Boyer-Lindquist coordinates on $M_{\rm I}$

Let us introduce on the chart of Boyer-Lindquist coordinates $(t,r,\theta,\varphi)$ on $M_{\rm I}$:

In [14]:
BL.<t,r,th,ph> = M_I.chart(r"t r:(m,+oo) th:(0,pi):\theta ph:(0,2*pi):periodic:\varphi") 
print(BL); BL

Chart (M_I, (t, r, th, ph))


Chart (M_I, (t, r, th, ph))

In [15]:
BL.coord_range()

t: (-oo, +oo); r: (m, +oo); th: (0, pi); ph: [0, 2*pi] (periodic)

In [16]:
KC_to_BL = KC.restrict(M_I).transition_map(BL, [tt + 2*m^2/(r-m) - 2*m*ln(abs(r-m)/m),
                                                r, th, tph + m/(r-m)])
KC_to_BL.display()

t = -2*m*log(abs(-m + r)/m) - 2*m^2/(m - r) + tt
r = r
th = th
ph = tph - m/(m - r)

In [17]:
KC_to_BL.inverse().display()

tt = -(2*m^2*log(m) - 2*m*r*log(m) - 2*m^2 - (m - r)*t - 2*(m^2 - m*r)*log(-m + r))/(m - r)
r = r
th = th
tph = (m*ph - ph*r + m)/(m - r)

In [18]:
g.display(BL)

g = -(m^2*cos(th)^2 - 2*m*r + r^2)/(m^2*cos(th)^2 + r^2) dt*dt - 2*m^2*r*sin(th)^2/(m^2*cos(th)^2 + r^2) dt*dph + (m^2*cos(th)^2 + r^2)/(m^2 - 2*m*r + r^2) dr*dr + (m^2*cos(th)^2 + r^2) dth*dth - 2*m^2*r*sin(th)^2/(m^2*cos(th)^2 + r^2) dph*dt + (2*m^3*r*sin(th)^4 + (m^2*r^2 + r^4 + (m^4 + m^2*r^2)*cos(th)^2)*sin(th)^2)/(m^2*cos(th)^2 + r^2) dph*dph

## Ingoing principal null geodesics

In [19]:
k = M.vector_field(1, -1, 0, 0, name='k')
k.display()

k = d/dtt - d/dr

Let us check that $k$ is a null vector:

In [20]:
g(k, k).expr()

0

Check that $k$ is a geodesic vector field, i.e. obeys $\nabla_k k = 0$:

In [21]:
nabla = g.connection()

In [22]:
nabla(k).contract(k).display()

0

Expression of $k$ with respect to the Boyer-Lindquist frame:

In [23]:
k.display(BL)

k = (m^2 + r^2)/(m^2 - 2*m*r + r^2) d/dt - d/dr + m/(m^2 - 2*m*r + r^2) d/dph

## Outgoing principal null geodesics

In [24]:
el = M.vector_field((r + m)^2/(2*(r^2 + m^2)),
                    (r - m)^2/(2*(r^2 + m^2)),
                    0,
                    m/(r^2 + m^2),
                    name='el', latex_name=r'\ell')
el.display()

el = 1/2*(m + r)^2/(m^2 + r^2) d/dtt + 1/2*(m - r)^2/(m^2 + r^2) d/dr + m/(m^2 + r^2) d/dtph

Let us check that $\ell$ is a null vector:

In [25]:
g(el, el).expr()

0

Expression of $\ell$ with respect to the Boyer-Lindquist frame:

In [26]:
el.display(BL)

el = 1/2 d/dt + 1/2*(m - r)^2/(m^2 + r^2) d/dr + 1/2*m/(m^2 + r^2) d/dph

Computation of $\nabla_\ell \ell$:

In [27]:
acc = nabla(el).contract(el)
acc.display()

-1/2*(m^5 + 2*m^4*r - 2*m^2*r^3 - m*r^4)/(m^6 + 3*m^4*r^2 + 3*m^2*r^4 + r^6) d/dtt - 1/2*(m^5 - 2*m^4*r + 2*m^2*r^3 - m*r^4)/(m^6 + 3*m^4*r^2 + 3*m^2*r^4 + r^6) d/dr - (m^4 - m^2*r^2)/(m^6 + 3*m^4*r^2 + 3*m^2*r^4 + r^6) d/dtph

We check that $\nabla_\ell \ell = \kappa \ell$:

In [28]:
kappa = acc[0] / el[0]
kappa

-(m^3 - m*r^2)/(m^4 + 2*m^2*r^2 + r^4)

In [29]:
kappa.factor()

-(m + r)*(m - r)*m/(m^2 + r^2)^2

In [30]:
acc == kappa*el

True

## Outgoing Kerr coordinates on $M_{\rm I}$

In [31]:
OKC.<to,r,th,oph> = M_I.chart(r"to:\tilde{\tilde{t}} r:(m,+oo) th:(0,pi):\theta oph:(0,2*pi):periodic:\tilde{\tilde{\varphi}}") 
OKC.coord_range()

to: (-oo, +oo); r: (m, +oo); th: (0, pi); oph: [0, 2*pi] (periodic)

In [32]:
BL_to_OKC = BL.transition_map(OKC, [t + 2*m^2/(r-m) - 2*m*ln(abs(r-m)/m),
                                    r, th, ph + m/(r-m)])
BL_to_OKC.display()

to = -2*m*log(abs(-m + r)/m) - 2*m^2/(m - r) + t
r = r
th = th
oph = ph - m/(m - r)

In [33]:
BL_to_OKC.inverse().display()

t = -(2*m^2*log(m) - 2*m*r*log(m) - 2*m^2 - (m - r)*to - 2*(m^2 - m*r)*log(-m + r))/(m - r)
r = r
th = th
ph = (m*oph - oph*r + m)/(m - r)

In [34]:
KC_to_OKC = BL_to_OKC * KC_to_BL.restrict(M_I)
KC_to_OKC.display()

to = (4*m^2*log(m) - 4*m*r*log(m) - 4*m^2 + (m - r)*tt - 4*(m^2 - m*r)*log(-m + r))/(m - r)
r = r
th = th
oph = ((m - r)*tph - 2*m)/(m - r)

In [35]:
KC_to_OKC.inverse().display()

tt = -(4*m^2*log(m) - 4*m*r*log(m) - 4*m^2 - (m - r)*to - 4*(m^2 - m*r)*log(-m + r))/(m - r)
r = r
th = th
tph = (m*oph - oph*r + 2*m)/(m - r)

In [36]:
M_I.set_default_chart(OKC)
M_I.set_default_frame(OKC.frame())

In [37]:
gI = g.restrict(M_I)
gI.display()

g = -(m^2*cos(th)^2 - 2*m*r + r^2)/(m^2*cos(th)^2 + r^2) dto*dto - 2*m*r/(m^2*cos(th)^2 + r^2) dto*dr - 2*m^2*r*sin(th)^2/(m^2*cos(th)^2 + r^2) dto*doph - 2*m*r/(m^2*cos(th)^2 + r^2) dr*dto + (m^2*cos(th)^2 + 2*m*r + r^2)/(m^2*cos(th)^2 + r^2) dr*dr - (m^3*sin(th)^4 - (m^3 + 2*m^2*r + m*r^2)*sin(th)^2)/(m^2*cos(th)^2 + r^2) dr*doph + (m^2*cos(th)^2 + r^2) dth*dth - 2*m^2*r*sin(th)^2/(m^2*cos(th)^2 + r^2) doph*dto - (m^3*sin(th)^4 - (m^3 + 2*m^2*r + m*r^2)*sin(th)^2)/(m^2*cos(th)^2 + r^2) doph*dr + (2*m^3*r*sin(th)^4 + (m^2*r^2 + r^4 + (m^4 + m^2*r^2)*cos(th)^2)*sin(th)^2)/(m^2*cos(th)^2 + r^2) doph*doph

In [38]:
gI[1,3]

-(m^3*sin(th)^4 - (m^3 + 2*m^2*r + m*r^2)*sin(th)^2)/(m^2*cos(th)^2 + r^2)

In [39]:
gI[1,3] == m*(1 + 2*m*r/rho2)*sin(th)^2

True

In [40]:
gI[3,3]

(2*m^3*r*sin(th)^4 + (m^2*r^2 + r^4 + (m^4 + m^2*r^2)*cos(th)^2)*sin(th)^2)/(m^2*cos(th)^2 + r^2)

In [41]:
g[3,3] == (r^2 + m^2 + 2*m^3*r*sin(th)^2/rho2)*sin(th)^2

True

In [42]:
ol = M_I.vector_field({OKC.frame(): (1, 1, 0, 0)}, name='ol', 
                      latex_name=r"\ell'")
ol.display()

ol = d/dto + d/dr

In [43]:
ol.display(KC.restrict(M_I).frame())

ol = (m^2 + 2*m*r + r^2)/(m^2 - 2*m*r + r^2) d/dtt + d/dr + 2*m/(m^2 - 2*m*r + r^2) d/dtph

In [44]:
ol.display(BL.frame())

ol = (m^2 + r^2)/(m^2 - 2*m*r + r^2) d/dt + d/dr + m/(m^2 - 2*m*r + r^2) d/dph

In [45]:
g(ol, ol).expr()

0

In [46]:
nabla.coef(OKC.frame())

3-indices components w.r.t. Coordinate frame (M_I, (d/dto,d/dr,d/dth,d/doph)), with symmetry on the index positions (1, 2)

In [47]:
nabla(ol).contract(ol).display()

0

In [48]:
elI = el.restrict(M_I)
elI.display()

el = 1/2*(m^2 - 2*m*r + r^2)/(m^2 + r^2) d/dto + 1/2*(m^2 - 2*m*r + r^2)/(m^2 + r^2) d/dr

Check of the relation $\ell' = 2 \frac{r^2 + m^2}{(r - m)^2} \, \ell$:

In [49]:
ol == 2*(r^2 + m^2)/(r - m)^2 * elI

True

In [50]:
kI = k.restrict(M_I)
kI.display()

k = (m^2 + 2*m*r + r^2)/(m^2 - 2*m*r + r^2) d/dto - d/dr + 2*m/(m^2 - 2*m*r + r^2) d/doph

In [51]:
ok = (r - m)^2/(2*(r^2 + m^2)) * kI
ok.set_name('ok', latex_name=r"k'")
ok.display()

ok = 1/2*(m^2 + 2*m*r + r^2)/(m^2 + r^2) d/dto - 1/2*(m^2 - 2*m*r + r^2)/(m^2 + r^2) d/dr + m/(m^2 + r^2) d/doph

In [52]:
g(k, el).expr()

-(m^2*cos(th)^2 + r^2)/(m^2 + r^2)

In [53]:
g(ok, ol).expr()

-(m^2*cos(th)^2 + r^2)/(m^2 + r^2)

In [54]:
g(k, ol).expr().factor()

-2*(m^2*cos(th)^2 + r^2)/(m - r)^2

In [55]:
g(ok, el).expr().factor()

-1/2*(m^2*cos(th)^2 + r^2)*(m - r)^2/(m^2 + r^2)^2

### Non-affinity coefficient of $k'$

In [56]:
acc_ok = nabla(ok).contract(ok)
acc_ok.display()

1/2*(m^5 + 2*m^4*r - 2*m^2*r^3 - m*r^4)/(m^6 + 3*m^4*r^2 + 3*m^2*r^4 + r^6) d/dto - 1/2*(m^5 - 2*m^4*r + 2*m^2*r^3 - m*r^4)/(m^6 + 3*m^4*r^2 + 3*m^2*r^4 + r^6) d/dr + (m^4 - m^2*r^2)/(m^6 + 3*m^4*r^2 + 3*m^2*r^4 + r^6) d/doph

In [57]:
kappa_ok = acc_ok[0] / ok[0]
kappa_ok.factor()

(m + r)*(m - r)*m/(m^2 + r^2)^2

We check that $\nabla_{k'} k' = \kappa_{k'} k'$:

In [58]:
acc_ok == kappa_ok * ok

True